In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as ply
train_data = pd.read_csv('train.csv').drop('Id', axis=1)

In [2]:
X_train = train_data.iloc[:, 1:].to_numpy()
y_train = train_data['y'].to_numpy()

In [3]:
def funky_transform(X):
    linear = X
    quadratic = X ** 2
    exponential = np.exp(X)
    cosine = np.cos(X)
    constant = np.ones(X.shape[0])
    constant = constant[:, np.newaxis]
    return np.hstack((linear, quadratic, exponential, cosine, constant))

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (GridSearchCV,
    cross_val_score, KFold)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import Ridge

pipeline = make_pipeline(FunctionTransformer(
                             funky_transform,
                             validate=True),
                         Ridge())
parameter_space = {
    'ridge__alpha': np.logspace(-5, 5, 100)
}
inner_cv = KFold(n_splits=10, shuffle=True)
outer_cv = KFold(n_splits=10, shuffle=True)
classifier = GridSearchCV(pipeline, parameter_space,
                               n_jobs=1, scoring='neg_mean_squared_error',
                               
                               refit=True,
                               cv=inner_cv)
scores = cross_val_score(classifier, X_train, y_train.ravel(),
                            cv=outer_cv,
                            scoring='neg_mean_squared_error',
                            n_jobs=1)
print("Cross-validation score is {score:.3f},"
      " standard deviation is {err:.3f}"
      .format(score = np.sqrt(-scores).mean(), err = np.sqrt(-scores).std()))

classifier = classifier.fit(X_train, y_train.ravel())
coefficients = classifier.best_estimator_['ridge'].coef_

Cross-validation score is 1.958, standard deviation is 0.137


In [6]:
display(coefficients)

array([-0.05430861, -0.71118115, -0.87831711, -0.05193159, -0.04808066,
       -0.27996084,  0.21553957,  0.17745964, -0.04309758,  0.12574522,
       -0.20400854, -0.60738281, -0.79833481, -0.08308559,  0.00307285,
        0.13912942, -0.10718517, -0.08812766,  0.02046101, -0.06091483,
        0.        ])